In [103]:
import pydeck as pdk
import pandas as pd
import numpy as np
import json

# Exploring Phantom Rides

In [2]:
phantom = pd.read_csv('./../data/phantom2.csv')

In [3]:
phantom.head(2)

,bike_id,end_station_id,end_station_name,end_lat,end_lon,end_time,next_station_id,next_station_name,next_lat,next_lon,next_time,end_borough,next_borough,end_day,next_day
0,16344,507,E 25 St & 2 Ave,40.739126,-73.979738,2013-06-09 20:40:52,164,E 47 St & 2 Ave,40.753231,-73.970325,2013-06-10 09:49:57,Manhattan,Manhattan,1,2
1,16344,510,W 51 St & 6 Ave,40.760660,-73.980420,2013-06-12 07:53:02,484,W 44 St & 5 Ave,40.755003,-73.980144,2013-06-13 17:29:33,Manhattan,Manhattan,4,5


# Weekday Rebalancing Rides


In [96]:
weekday_phantom = phantom[phantom['end_day'].isin([2,3,4,5,6])]

In [97]:
weekday_grouped = weekday_phantom.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [98]:
weekday_grouped.sort_values(by = 'count',ascending = False, inplace = True)

In [104]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekday_grouped[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/700')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekday_rebalancing.html")
with open('weekday_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)


# Weekend Rebalancing Rides

In [8]:
weekend_phantom = phantom[phantom['end_day'].isin([1,7])]

In [9]:
weekend_grouped = weekend_phantom.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [10]:
weekend_grouped.sort_values(by = 'count',ascending = False, inplace = True)

In [95]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekend_grouped[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/100')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekend_rebalancing.html")

# Lets get a bit more refined
We will start by lookig at the rebalancing that happens in the AM of the weedays

In [28]:
phantom3 = pd.read_parquet('./../data/phantom3.parquet')

In [29]:
phantom3.head()

,bike_id,end_station_id,end_station_name,end_lat,end_lon,end_time,next_station_id,next_station_name,next_lat,next_lon,next_time,end_borough,next_borough,end_day,next_day,midpoint
0,16344,507,E 25 St & 2 Ave,40.739126,-73.979738,2013-06-09 20:40:52,164,E 47 St & 2 Ave,40.753231,-73.970325,2013-06-10 09:49:57,Manhattan,Manhattan,1,2,2013-06-10 03:15:52
1,16344,510,W 51 St & 6 Ave,40.760660,-73.980420,2013-06-12 07:53:02,484,W 44 St & 5 Ave,40.755003,-73.980144,2013-06-13 17:29:33,Manhattan,Manhattan,4,5,2013-06-13 00:41:02
2,16344,487,E 20 St & FDR Drive,40.733143,-73.975739,2013-06-15 14:42:09,356,Bialystoker Pl & Delancey St,40.716226,-73.982612,2013-06-15 17:42:42,Manhattan,Manhattan,7,7,2013-06-15 16:12:09
3,16344,517,Pershing Square South,40.751581,-73.977910,2013-06-21 17:44:13,533,Broadway & W 38 St,40.752996,-73.987216,2013-06-22 08:06:35,Manhattan,Manhattan,6,7,2013-06-22 00:55:13
4,16344,521,8 Ave & W 31 St N,40.750967,-73.994442,2013-06-23 21:50:45,531,Forsyth St & Broome St,40.718939,-73.992663,2013-06-25 12:21:56,Manhattan,Manhattan,1,3,2013-06-24 17:06:45


In [90]:
weekday_morning = phantom3[(phantom3['end_day'] == phantom3['next_day']) &\
                           phantom3['midpoint'].dt.dayofweek.isin([0,6]) &\
                          phantom3['midpoint'].dt.hour.isin([23,0,1,2,3])]

In [91]:
weekday_morning = weekday_morning.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [92]:
weekday_morning.sort_values(by = 'count', ascending = False,inplace = True)

In [93]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekday_morning[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/2')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekend_rebalancing.html")

## Now weekdays midday